In [1]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

def article_parsing(url):
    # request error를 피하기 위한 header 설정
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

    # html parser
    web = BeautifulSoup(requests.get(url, allow_redirects=False, headers=header).text, 'html.parser')

    # 제목
    title = web.find("meta", property="og:title")['content']

    # 요약 내용
    description = web.find('meta', property="og:description")['content']
    
    # 대표 이미지 url
    image_url = web.find("meta", property="og:image")['content']

    # 대표이미지 객체
    image = Image.open(BytesIO(requests.get(image_url, headers=header).content))
    
    return {'title':title, 'description' : description, 'image': image, 'image_url':image_url, 'url':url }

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 크롤링할 네이버 뉴스 URL 설정
url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=%EC%9E%AC%ED%99%9C%EC%9A%A9'

# 네이버 뉴스 페이지 요청
response = requests.get(url)

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 기사 제목과 링크 추출
articles = []
for article in soup.find_all('a', class_='news_tit'):
    title = article.get_text().strip()  # 기사 제목
    link = article['href']  # 기사 링크
    articles.append({'Title': title, 'Link': link})

# 추출한 데이터를 DataFrame으로 변환
df = pd.DataFrame(articles)

# DataFrame을 CSV 파일로 저장
df.to_csv('E:/PycharmProjects/peslab/GDSC SC/naver_news_articles.csv', index=False, encoding='utf-8-sig')  # index는 저장하지 않음

print("크롤링 및 저장이 완료되었습니다.")


크롤링 및 저장이 완료되었습니다.


In [8]:
df

,Title,Link
0,서울 4400여개 어린이집 배출 '우유팩→노트·스케치북' 재활용,https://www.news1.kr/articles/5305530
1,"[서울] 서울 시내 어린이집에서 쓴 우유팩, 학용품으로 재활용",https://www.ytn.co.kr/_ln/0115_202401301552380070
2,"KCC, 재활용‧친환경 디자인 표준 도입…ESG 경영 박차",https://sports.hankooki.com/news/articleView.h...
3,"롯데百 '보랭가방', 버리지 말고 재활용…엘포인트 5000점 지급",http://news.mt.co.kr/mtview.php?no=20240130084...
4,"불발탄 해체해 재활용...하마스 미사일·로켓 상당수, 이스라엘서 왔다",https://www.chosun.com/international/mideast-a...
5,"청주시, 11월까지 아파트·학교 대상 재활용품 수집 대회",https://www.yna.co.kr/view/AKR2024012905440006...
6,"서대문구, AI 무인회수기 설치…""재활용도 하고 돈도 받고""[동네방네]",http://www.edaily.co.kr/news/newspath.asp?news...
7,"주얼리 ‘판도라’, 채굴 대신 100% 재활용 금·은 쓴다 [포토]",https://www.hani.co.kr/arti/international/euro...
8,"환경장관, '처치 곤란' 태양광 폐패널 자원 재활용 현장 찾는다",http://www.newsis.com/view/?id=NISX20240129_00...
9,"'그린슈머' 잡는 음료업계 친환경 바람... 브리타, 정수기 필터 재활용",https://www.newspim.com/news/view/20240130000805


In [13]:
df.to_csv('E:/PycharmProjects/peslab/GDSC SC/naver_news_articles.csv', index=False, encoding='utf-8-sig')  # index는 저장하지 않음

## 위의 코드를 실시간으로 크롤링해서 저장해주는 코드 작성해야함

In [9]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import schedule
import time

# 초기 CSV 파일 생성 (최초 1회만 실행)
initial_url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=%EC%9E%AC%ED%99%9C%EC%9A%A9'
initial_response = requests.get(initial_url)
initial_soup = BeautifulSoup(initial_response.text, 'html.parser')
initial_articles = []

for article in initial_soup.find_all('a', class_='news_tit'):
    title = article.get_text().strip()  # 기사 제목
    link = article['href']  # 기사 링크
    initial_articles.append({'Title': title, 'Link': link})

initial_df = pd.DataFrame(initial_articles)
initial_df.to_csv('E:/PycharmProjects/peslab/GDSC SC/naver_news_articles.csv', index=False, encoding='utf-8-sig')

def crawl_and_append():
    # 크롤링할 네이버 뉴스 URL 설정
    url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=%EC%9E%AC%ED%99%9C%EC%9A%A9'

    # 네이버 뉴스 페이지 요청
    response = requests.get(url)

    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 기사 제목과 링크 추출
    articles = []
    for article in soup.find_all('a', class_='news_tit'):
        title = article.get_text().strip()  # 기사 제목
        link = article['href']  # 기사 링크
        articles.append({'Title': title, 'Link': link})

    # 기존 CSV 파일 읽기
    existing_df = pd.read_csv('E:/PycharmProjects/peslab/GDSC SC/naver_news_articles.csv', encoding='utf-8-sig')

    # 새로운 데이터를 기존 데이터에 추가
    updated_df = existing_df.append(articles, ignore_index=True)

    # 업데이트된 데이터를 CSV 파일로 저장
    updated_df.to_csv('E:/PycharmProjects/peslab/GDSC SC/naver_news_articles.csv', index=False, encoding='utf-8-sig')

    print("크롤링 및 추가 저장이 완료되었습니다.")

# 10분마다 크롤링 함수를 실행하는 스케줄 설정
schedule.every(10).minutes.do(crawl_and_append)

while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: 

In [12]:
df

,Title,Link
0,서울 4400여개 어린이집 배출 '우유팩→노트·스케치북' 재활용,https://www.news1.kr/articles/5305530
1,"[서울] 서울 시내 어린이집에서 쓴 우유팩, 학용품으로 재활용",https://www.ytn.co.kr/_ln/0115_202401301552380070
2,"KCC, 재활용‧친환경 디자인 표준 도입…ESG 경영 박차",https://sports.hankooki.com/news/articleView.h...
3,"롯데百 '보랭가방', 버리지 말고 재활용…엘포인트 5000점 지급",http://news.mt.co.kr/mtview.php?no=20240130084...
4,"불발탄 해체해 재활용...하마스 미사일·로켓 상당수, 이스라엘서 왔다",https://www.chosun.com/international/mideast-a...
5,"청주시, 11월까지 아파트·학교 대상 재활용품 수집 대회",https://www.yna.co.kr/view/AKR2024012905440006...
6,"서대문구, AI 무인회수기 설치…""재활용도 하고 돈도 받고""[동네방네]",http://www.edaily.co.kr/news/newspath.asp?news...
7,"주얼리 ‘판도라’, 채굴 대신 100% 재활용 금·은 쓴다 [포토]",https://www.hani.co.kr/arti/international/euro...
8,"환경장관, '처치 곤란' 태양광 폐패널 자원 재활용 현장 찾는다",http://www.newsis.com/view/?id=NISX20240129_00...
9,"'그린슈머' 잡는 음료업계 친환경 바람... 브리타, 정수기 필터 재활용",https://www.newspim.com/news/view/20240130000805


In [ ]:
df.csv()

In [1]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [2]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [3]:
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=91', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=101', 'https://search.naver.com/search.naver?where=n

100%|██████████| 145/145 [00:00<?, ?it/s]


In [5]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 55/55 [00:20<00:00,  2.70it/s]

검색된 기사 갯수: 총  200 개

[뉴스 제목]
['정읍 폐어망재활용 시설서 화재…대응 1단계 발령', '청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치', '전북 정읍 재활용업체서 불…한때 ‘대응 1단계’ 발령', "'면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증", '전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”', '하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실', '정읍 재활용시설 공장서 불…대응 1단계 발령', '콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차', '"재활용 박스로 친환경"…오아시스, 택배 박스 수거 나선다', '중금속 재활용업체 황조, 몸값 1000억대에 매각한다', '하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실', '정읍 재활용업체 불…큰 불길은 잡혀', '아이오닉 690만원, 레이 452만원…올해 전기차 차종별 보조금 확정', '車운송은 기본 … 배터리 재활용 금맥캔다', '다 쓴 리튬 일차전지 재활용 친환경 기술 개발', '아모레퍼시픽그룹, 화장품 유리병 재활용 위한 협력 체계 구축', "나경수 SK지오센트릭 사장, '바이바이 플라스틱 챌린지' 동참", '아모레퍼시픽그룹, 한국순환자원유통지원센터와 맞손 "유리병 재활용"', '용인시, 폐가전제품 재활용 일원화…전국 최초 처리시스템 도입', '전남도, 버려질 수목 ‘나무은행’ 통해 재활용…17억 절감', "최태원 회장 지목 나경수 사장, 'ByeBye 플라스틱' 동참", "마포구, 전국 첫 쓰레기 감량정책 '폐기물 조례' 만든다", '인천시 "재활용품 현금보상 자원순환가게, 99개소 확대"', '청주재활용센터, 매월 셋째 수요일 무료 가구 수리', '경기도, 영농폐기물 집중 수거기간…수거보상금 지급', "청주시, 복지 사각지대 예방 '폐지 줍는 노인' 전수조사", '부천시, 재활용품 수집 노인 전수조사 나서', '이천 플라스틱 생산공장 화재…14명 대피'

In [6]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  54


In [9]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

def article_parsing(url):
    # request error를 피하기 위한 header 설정
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

    # html parser
    web = BeautifulSoup(requests.get(url, allow_redirects=False, headers=header).text, 'html.parser')

    # 제목
    title = web.find("meta", property="og:title")['content']

    # 요약 내용
    description = web.find('meta', property="og:description")['content']
    
    # 대표 이미지 url
    image_url = web.find("meta", property="og:image")['content']

    # 대표이미지 객체
    image = Image.open(BytesIO(requests.get(image_url, headers=header).content))
    
    return {'title':title, 'description' : description, 'image': image, 'image_url':image_url, 'url':url }


In [28]:
final_urls

['https://n.news.naver.com/mnews/article/001/0014514421?sid=102',
 'https://n.news.naver.com/mnews/article/003/0012381285?sid=102',
 'https://n.news.naver.com/mnews/article/056/0011665157?sid=102',
 'https://n.news.naver.com/mnews/article/011/0004302087?sid=101',
 'https://n.news.naver.com/mnews/article/016/0002269137?sid=105',
 'https://n.news.naver.com/mnews/article/021/0002622212?sid=101',
 'https://n.news.naver.com/mnews/article/003/0012381210?sid=102',
 'https://n.news.naver.com/mnews/article/020/0003548931?sid=103',
 'https://n.news.naver.com/mnews/article/648/0000023411?sid=101',
 'https://n.news.naver.com/mnews/article/015/0004950140?sid=101',
 'https://n.news.naver.com/mnews/article/021/0002622212?sid=101',
 'https://n.news.naver.com/mnews/article/056/0011665232?sid=102',
 'https://n.news.naver.com/mnews/article/421/0007361860?sid=101',
 'https://n.news.naver.com/mnews/article/009/0005260046?sid=103',
 'https://n.news.naver.com/mnews/article/001/0014507977?sid=101',
 'https://

In [30]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

def article_parsing(url):
    # request error를 피하기 위한 header 설정
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    
    # html parser
    web = BeautifulSoup(requests.get(url, allow_redirects=False, headers=header).text, 'html.parser')

    # 제목
    title = web.find("meta", property="og:title")['content'] if web.find("meta", property="og:title") else 'No Title Found'

    # 요약 내용
    description = web.find('meta', property="og:description")['content'] if web.find('meta', property="og:description") else 'No Description Found'
    
    # 대표 이미지 url
    image_url = web.find("meta", property="og:image")['content'] if web.find("meta", property="og:image") else 'No Image URL Found'

    # 대표이미지 객체
    try:
        image = Image.open(BytesIO(requests.get(image_url, headers=header).content))
    except:
        image = None  # 이미지 로드 실패 시 None 반환

    return {'title': title, 'description': description, 'image': image, 'image_url': image_url, 'url': url}


# 결과를 저장할 리스트
articles_data = []

# final_urls의 각 URL에 대해 함수 호출 및 결과 저장
for url in final_urls:
    article_data = article_parsing(url)
    articles_data.append(article_data)

# 결과 출력 (예시로, 여기서는 각 기사의 제목만 출력)
for article in articles_data:
    print(article['title'])


C:\Users\20161\AppData\Local\Temp\ipykernel_12916\4148350810.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  web = BeautifulSoup(requests.get(url, allow_redirects=False, headers=header).text, 'html.parser')


정읍 폐어망재활용 시설서 화재…대응 1단계 발령
청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치
전북 정읍 재활용업체서 불…한때 ‘대응 1단계’ 발령
'면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증
전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”
하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실
정읍 재활용시설 공장서 불…대응 1단계 발령
콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차
"재활용 박스로 친환경"…오아시스, 택배 박스 수거 나선다
중금속 재활용업체 황조, 몸값 1000억대에 매각한다
하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실
정읍 재활용업체 불…큰 불길은 잡혀
아이오닉 690만원, 레이 452만원…올해 전기차 차종별 보조금 확정
車운송은 기본 … 배터리 재활용 금맥캔다
다 쓴 리튬 일차전지 재활용 친환경 기술 개발
아모레퍼시픽그룹, 화장품 유리병 재활용 위한 협력 체계 구축
나경수 SK지오센트릭 사장, '바이바이 플라스틱 챌린지' 동참
아모레퍼시픽그룹, 한국순환자원유통지원센터와 맞손 "유리병 재활용"
용인시, 폐가전제품 재활용 일원화…전국 최초 처리시스템 도입
전남도, 버려질 수목 ‘나무은행’ 통해 재활용…17억 절감
최태원 회장 지목 나경수 사장, 'ByeBye 플라스틱' 동참
마포구, 전국 첫 쓰레기 감량정책 '폐기물 조례' 만든다
인천시 "재활용품 현금보상 자원순환가게, 99개소 확대"
청주재활용센터, 매월 셋째 수요일 무료 가구 수리
경기도, 영농폐기물 집중 수거기간…수거보상금 지급
청주시, 복지 사각지대 예방 '폐지 줍는 노인' 전수조사
부천시, 재활용품 수집 노인 전수조사 나서
이천 플라스틱 생산공장 화재…14명 대피
EU 배터리법 18일 시행…2031년부터 리튬·니켈 등 재활용 의무
[밀착카메라] '양심까지 버렸나?' 연휴 뒤 쓰레기, 재활용 불가도 '산더미'
깨끗한나라, 스티로폼 재활용 'EPS 마이크로펠릿' 개

In [ ]:
articles_data

In [ ]:
import pandas as pd

# articles_data 리스트가 이미 위의 코드를 통해 채워져 있다고 가정

# 이미지 객체 대신 이미지 URL을 사용하여 데이터프레임을 생성
df_articles = pd.DataFrame(articles_data).drop('image', axis=1)

# 데이터프레임 확인
print(df_articles)

# 필요한 경우 CSV 파일로 저장
df_articles.to_csv('C:/Users/20161/Downloads/articles_data.csv', index=False)


# 기사 게시 날짜도 추출하기

In [54]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
from datetime import datetime

def article_parsing(url):
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    
    web = BeautifulSoup(requests.get(url, allow_redirects=False, headers=header).text, 'html.parser')

    title = web.find("meta", property="og:title")['content'] if web.find("meta", property="og:title") else 'No Title Found'
    description = web.find('meta', property="og:description")['content'] if web.find('meta', property="og:description") else 'No Description Found'
    image_url = web.find("meta", property="og:image")['content'] if web.find("meta", property="og:image") else 'No Image URL Found'
    try:
        image = Image.open(BytesIO(requests.get(image_url, headers=header).content))
    except:
        image = None

    # 기사 게시 날짜가 크롤링되지 않는 경우 현재 시간 사용
    publish_time = datetime.now().strftime('%Y-%m-%d')

    return {'title': title, 'description': description, 'image': image, 'image_url': image_url, 'publish_time': publish_time, 'url': url}

# 결과를 저장할 리스트
articles_data = []

# final_urls의 각 URL에 대해 함수 호출 및 결과 저장
for url in final_urls:
    article_data = article_parsing(url)
    articles_data.append(article_data)

# 결과 출력 (예시로, 여기서는 각 기사의 제목만 출력)
for article in articles_data:
    print(article['title'])


C:\Users\20161\AppData\Local\Temp\ipykernel_12916\4161095017.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  web = BeautifulSoup(requests.get(url, allow_redirects=False, headers=header).text, 'html.parser')


테슬라 모델Y, 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓
청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치
'면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증
정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해
전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”
하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실
정읍 재활용시설 공장서 불…대응 1단계 발령
콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차
울진군, 자원재활용사업으로 탄소중립도시 실천
부산시설공단, 녹지대 폐목재 재활용…농가에 무상 공급
전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”
중금속 재활용업체 황조, 몸값 1000억대에 매각한다
하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실
정읍 재활용업체 불…큰 불길은 잡혀
車운송은 기본 … 배터리 재활용 금맥캔다
다 쓴 리튬 일차전지 재활용 친환경 기술 개발
아모레퍼시픽그룹, 화장품 유리병 재활용 위한 협력 체계 구축
테슬라 모델Y 195만원 받을 때 현대 아이오닉6 690만원
최태원 회장 지목 나경수 사장, 'ByeBye 플라스틱' 동참
아모레퍼시픽그룹, 한국순환자원유통지원센터와 맞손 "유리병 재활용"
용인시, 폐가전제품 재활용 일원화…전국 최초 처리시스템 도입
전남도, 버려질 수목 ‘나무은행’ 통해 재활용…17억 절감
나경수 SK지오센트릭 사장, '바이바이 플라스틱 챌린지' 동참
인천시 "재활용품 현금보상 자원순환가게, 99개소 확대"
마포구, 전국 첫 쓰레기 감량정책 '폐기물 조례' 만든다
청주재활용센터, 매월 셋째 수요일 무료 가구 수리
경기도, 영농폐기물 집중 수거기간…수거보상금 지급
청주시, 복지 사각지대 예방 '폐지 줍는 노인' 전수조사
이천 플라스틱 생산공장 화재…14명 대피
부천시, 재활용품 수집 노인 전수조사 나서
EU 배터리법 18일 시행…2031년부터 리튬·니켈 등 재활용 의무
[밀착카메라] 

In [55]:
import pandas as pd

# articles_data 리스트가 이미 위의 코드를 통해 채워져 있다고 가정

# 이미지 객체 대신 이미지 URL을 사용하여 데이터프레임을 생성
df_articles = pd.DataFrame(articles_data).drop('image', axis=1)

# 데이터프레임 확인
print(df_articles)

# 필요한 경우 CSV 파일로 저장
df_articles.to_csv('C:/Users/20161/Downloads/articles_data_new.csv', index=False)


                                           title  \
0           테슬라 모델Y, 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓   
1                청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치   
2              '면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증   
3               정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해   
4            전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”   
5     하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실   
6                       정읍 재활용시설 공장서 불…대응 1단계 발령   
7        콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차   
8                       울진군, 자원재활용사업으로 탄소중립도시 실천   
9                  부산시설공단, 녹지대 폐목재 재활용…농가에 무상 공급   
10           전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”   
11                 중금속 재활용업체 황조, 몸값 1000억대에 매각한다   
12    하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실   
13                           정읍 재활용업체 불…큰 불길은 잡혀   
14                        車운송은 기본 … 배터리 재활용 금맥캔다   
15                     다 쓴 리튬 일차전지 재활용 친환경 기술 개발   
16             아모레퍼시픽그룹, 화장품 유리병 재활용 위한 협력 체계 구축   
17             테슬라 모델Y 195만원 받을 때 현대 아이오닉6 690만원   
18          

In [56]:
df_articles

,title,description,image_url,publish_time,url
0,"테슬라 모델Y, 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓",환경부가 20일 전기차 보조금 업무처리지침을 확정하고 차종별 보조금 지원 금액을 공...,https://imgnews.pstatic.net/image/025/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/025/000...
1,"청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치",충북 청주시는 소규모 주택가에 재활용품 분리수거함을 설치 지원한다고 20일 밝혔다....,https://imgnews.pstatic.net/image/003/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/003/001...
2,"'면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증",기후테크 기업 제클린은 독자 개발한 재생 면화가 업계 최초의 우수 재활용 제품(GR...,https://imgnews.pstatic.net/image/011/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/011/000...
3,정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해,오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망 재활용 업체에서 불이 나 5시간여...,https://imgnews.pstatic.net/image/056/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/056/001...
4,전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”,"전기차 폭발, 유독성 화학물질 유출 등 배터리 안전문제가 대두되면서 범부처 ‘사용후...",https://imgnews.pstatic.net/image/016/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/016/000...
5,하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실,‘소셜벤처 유니버시티’선순환 세계적 수준의 쓰레기 분리수거 제도를 시행하고 있는 우...,https://imgnews.pstatic.net/image/021/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/021/000...
6,정읍 재활용시설 공장서 불…대응 1단계 발령,20일 오전 5시 전북 정읍시 소성면 용정리의 한 재활용시설 공장서 불이 났다. 소...,https://imgnews.pstatic.net/image/003/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/003/001...
7,"콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차",콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 전략을 공개했다. 현...,https://imgnews.pstatic.net/image/020/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/020/000...
8,"울진군, 자원재활용사업으로 탄소중립도시 실천",경북 울진군은 다양한 자원재활용 사업을 추진해 탄소중립 도시 정책에 군민들이 발맞추...,https://imgnews.pstatic.net/image/277/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/277/000...
9,"부산시설공단, 녹지대 폐목재 재활용…농가에 무상 공급",부산시설공단은 도시고속도로 번영로 녹지대 관리 과정에서 발생하는 소나무 목재조각(우...,https://imgnews.pstatic.net/image/001/2024/02/...,2024-02-20,https://n.news.naver.com/mnews/article/001/001...


In [57]:
# DataFrame 컬럼 순서 및 이름 변경
df_articles = df_articles[['publish_time', 'title', 'url', 'description', 'image_url']]
df_articles.columns = ['date', 'title', 'link', 'content', 'imageUrl']

# 변경된 DataFrame 저장
df_articles.to_csv('C:/Users/20161/Downloads/articles_data_updated.csv', index=False, encoding='utf-8-sig')
df_articles


,date,title,link,content,imageUrl
0,2024-02-20,"테슬라 모델Y, 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓",https://n.news.naver.com/mnews/article/025/000...,환경부가 20일 전기차 보조금 업무처리지침을 확정하고 차종별 보조금 지원 금액을 공...,https://imgnews.pstatic.net/image/025/2024/02/...
1,2024-02-20,"청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치",https://n.news.naver.com/mnews/article/003/001...,충북 청주시는 소규모 주택가에 재활용품 분리수거함을 설치 지원한다고 20일 밝혔다....,https://imgnews.pstatic.net/image/003/2024/02/...
2,2024-02-20,"'면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증",https://n.news.naver.com/mnews/article/011/000...,기후테크 기업 제클린은 독자 개발한 재생 면화가 업계 최초의 우수 재활용 제품(GR...,https://imgnews.pstatic.net/image/011/2024/02/...
3,2024-02-20,정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해,https://n.news.naver.com/mnews/article/056/001...,오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망 재활용 업체에서 불이 나 5시간여...,https://imgnews.pstatic.net/image/056/2024/02/...
4,2024-02-20,전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”,https://n.news.naver.com/mnews/article/016/000...,"전기차 폭발, 유독성 화학물질 유출 등 배터리 안전문제가 대두되면서 범부처 ‘사용후...",https://imgnews.pstatic.net/image/016/2024/02/...
5,2024-02-20,하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실,https://n.news.naver.com/mnews/article/021/000...,‘소셜벤처 유니버시티’선순환 세계적 수준의 쓰레기 분리수거 제도를 시행하고 있는 우...,https://imgnews.pstatic.net/image/021/2024/02/...
6,2024-02-20,정읍 재활용시설 공장서 불…대응 1단계 발령,https://n.news.naver.com/mnews/article/003/001...,20일 오전 5시 전북 정읍시 소성면 용정리의 한 재활용시설 공장서 불이 났다. 소...,https://imgnews.pstatic.net/image/003/2024/02/...
7,2024-02-20,"콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차",https://n.news.naver.com/mnews/article/020/000...,콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 전략을 공개했다. 현...,https://imgnews.pstatic.net/image/020/2024/02/...
8,2024-02-20,"울진군, 자원재활용사업으로 탄소중립도시 실천",https://n.news.naver.com/mnews/article/277/000...,경북 울진군은 다양한 자원재활용 사업을 추진해 탄소중립 도시 정책에 군민들이 발맞추...,https://imgnews.pstatic.net/image/277/2024/02/...
9,2024-02-20,"부산시설공단, 녹지대 폐목재 재활용…농가에 무상 공급",https://n.news.naver.com/mnews/article/001/001...,부산시설공단은 도시고속도로 번영로 녹지대 관리 과정에서 발생하는 소나무 목재조각(우...,https://imgnews.pstatic.net/image/001/2024/02/...


In [58]:
df_articles['link'] = df_articles['link'].str.replace(',', '')
df_articles['content'] = df_articles['content'].str.replace(',', '')
df_articles['imageUrl'] = df_articles['imageUrl'].str.replace(',', '')
df_articles['date'] = df_articles['date'].str.replace(',', '')
df_articles['title'] = df_articles['title'].str.replace(',', '')

In [59]:
df_articles

,date,title,link,content,imageUrl
0,2024-02-20,테슬라 모델Y 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓,https://n.news.naver.com/mnews/article/025/000...,환경부가 20일 전기차 보조금 업무처리지침을 확정하고 차종별 보조금 지원 금액을 공...,https://imgnews.pstatic.net/image/025/2024/02/...
1,2024-02-20,청주시 소규모 주택가 66곳에 재활용품 분리수거함 설치,https://n.news.naver.com/mnews/article/003/001...,충북 청주시는 소규모 주택가에 재활용품 분리수거함을 설치 지원한다고 20일 밝혔다....,https://imgnews.pstatic.net/image/003/2024/02/...
2,2024-02-20,'면사의 끝없는 부활' 제클린 업계 최초 우수 재활용 인증,https://n.news.naver.com/mnews/article/011/000...,기후테크 기업 제클린은 독자 개발한 재생 면화가 업계 최초의 우수 재활용 제품(GR...,https://imgnews.pstatic.net/image/011/2024/02/...
3,2024-02-20,정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해,https://n.news.naver.com/mnews/article/056/001...,오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망 재활용 업체에서 불이 나 5시간여...,https://imgnews.pstatic.net/image/056/2024/02/...
4,2024-02-20,전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”,https://n.news.naver.com/mnews/article/016/000...,전기차 폭발 유독성 화학물질 유출 등 배터리 안전문제가 대두되면서 범부처 ‘사용후 ...,https://imgnews.pstatic.net/image/016/2024/02/...
5,2024-02-20,하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실,https://n.news.naver.com/mnews/article/021/000...,‘소셜벤처 유니버시티’선순환 세계적 수준의 쓰레기 분리수거 제도를 시행하고 있는 우...,https://imgnews.pstatic.net/image/021/2024/02/...
6,2024-02-20,정읍 재활용시설 공장서 불…대응 1단계 발령,https://n.news.naver.com/mnews/article/003/001...,20일 오전 5시 전북 정읍시 소성면 용정리의 한 재활용시설 공장서 불이 났다. 소...,https://imgnews.pstatic.net/image/003/2024/02/...
7,2024-02-20,콘티넨탈 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차,https://n.news.naver.com/mnews/article/020/000...,콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 전략을 공개했다. 현...,https://imgnews.pstatic.net/image/020/2024/02/...
8,2024-02-20,울진군 자원재활용사업으로 탄소중립도시 실천,https://n.news.naver.com/mnews/article/277/000...,경북 울진군은 다양한 자원재활용 사업을 추진해 탄소중립 도시 정책에 군민들이 발맞추...,https://imgnews.pstatic.net/image/277/2024/02/...
9,2024-02-20,부산시설공단 녹지대 폐목재 재활용…농가에 무상 공급,https://n.news.naver.com/mnews/article/001/001...,부산시설공단은 도시고속도로 번영로 녹지대 관리 과정에서 발생하는 소나무 목재조각(우...,https://imgnews.pstatic.net/image/001/2024/02/...


In [60]:

# 변경된 DataFrame 저장 (새로운 파일 경로로 저장)
new_file_path = 'C:/Users/20161/Downloads/articles_data_updated22.csv'  # 새 파일 경로로 변경
df_articles.to_csv(new_file_path, index=False, encoding='utf-8-sig')

print("CSV 파일이 성공적으로 처리되어 저장되었습니다.")

CSV 파일이 성공적으로 처리되어 저장되었습니다.


## 네이버 기사 content 크롤링

In [48]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [49]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=91', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=재활용&start=101', 'https://search.naver.com/search.naver?where=n

100%|██████████| 145/145 [00:00<?, ?it/s]


In [50]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 55/55 [00:33<00:00,  1.66it/s]

검색된 기사 갯수: 총  200 개

[뉴스 제목]
['테슬라 모델Y, 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓', '청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치', "'면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증", '정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해', '전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”', '하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실', '정읍 재활용시설 공장서 불…대응 1단계 발령', '콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차', '울진군, 자원재활용사업으로 탄소중립도시 실천', '부산시설공단, 녹지대 폐목재 재활용…농가에 무상 공급', '전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”', '중금속 재활용업체 황조, 몸값 1000억대에 매각한다', '하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실', '정읍 재활용업체 불…큰 불길은 잡혀', '車운송은 기본 … 배터리 재활용 금맥캔다', '다 쓴 리튬 일차전지 재활용 친환경 기술 개발', '아모레퍼시픽그룹, 화장품 유리병 재활용 위한 협력 체계 구축', '테슬라 모델Y 195만원 받을 때 현대 아이오닉6 690만원', "최태원 회장 지목 나경수 사장, 'ByeBye 플라스틱' 동참", '아모레퍼시픽그룹, 한국순환자원유통지원센터와 맞손 "유리병 재활용"', '용인시, 폐가전제품 재활용 일원화…전국 최초 처리시스템 도입', '전남도, 버려질 수목 ‘나무은행’ 통해 재활용…17억 절감', "나경수 SK지오센트릭 사장, '바이바이 플라스틱 챌린지' 동참", '인천시 "재활용품 현금보상 자원순환가게, 99개소 확대"', "마포구, 전국 첫 쓰레기 감량정책 '폐기물 조례' 만든다", '청주재활용센터, 매월 셋째 수요일 무료 가구 수리', '경기도, 영농폐기물 집중 수거기간…수거보상금 지급', "청주시, 복지 사각지대 

In [52]:
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
news_df.to_csv('C:/Users/20161/Downloads/articles_data_updated2.csv',encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  53


In [53]:
news_df

,date,title,link,content
0,2024-02-20 15:38:25,"테슬라 모델Y, 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓",https://n.news.naver.com/mnews/article/025/000...,[\n \n\n\n\n현대차 인도네시아 공장에서 전기차 '아이오닉 5'가 생산되고...
1,2024-02-20 08:39:34,"청주시, 소규모 주택가 66곳에 재활용품 분리수거함 설치",https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[청주=뉴시스] 충북 청주시청 제1임시청사. photo@news...
2,2024-02-20 12:08:01,"'면사의 끝없는 부활' 제클린, 업계 최초 우수 재활용 인증",https://n.news.naver.com/mnews/article/011/000...,[\n\n\n\n\nGR인증 재생 면사로 제작된 제클린의 '리[Re;] 타올'/사진...
3,2024-02-20 15:49:19,정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해,https://n.news.naver.com/mnews/article/056/001...,[\n\n\n\n\n[KBS 전주]오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망...
4,2024-02-20 11:32:03,전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”,https://n.news.naver.com/mnews/article/016/000...,"[\n\n\n\n\n전기차.[게티이미지뱅크][헤럴드경제=구본혁 ] 전기차 폭발, 유..."
5,2024-02-20 11:46:30,하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실,https://n.news.naver.com/mnews/article/021/000...,[\n\n\n\n\n‘소셜벤처 유니버시티’선순환세계적 수준의 쓰레기 분리수거 제도를...
6,2024-02-20 06:33:40,정읍 재활용시설 공장서 불…대응 1단계 발령,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n전북소방본부 제공 *재판매 및 DB 금지[정읍=뉴시스]최정규 기...
7,2024-02-20 10:19:01,"콘티넨탈, 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차",https://n.news.naver.com/mnews/article/020/000...,[\n\n\n\n\n콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 ...
8,2024-02-20 13:26:10,"울진군, 자원재활용사업으로 탄소중립도시 실천",https://n.news.naver.com/mnews/article/277/000...,[\n친환경 분리 배출 시스템으로 자원재활용 활성화 도모재활용품 자원화 시민참여 유...
9,2024-02-19 08:45:31,"부산시설공단, 녹지대 폐목재 재활용…농가에 무상 공급",https://n.news.naver.com/mnews/article/001/001...,"[\n\n\n\n\n부산시설공단, 녹지대 폐목재 재활용[부산시설공단 제공] (..."


In [61]:
df_articles

,date,title,link,content,imageUrl
0,2024-02-20,테슬라 모델Y 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓,https://n.news.naver.com/mnews/article/025/000...,환경부가 20일 전기차 보조금 업무처리지침을 확정하고 차종별 보조금 지원 금액을 공...,https://imgnews.pstatic.net/image/025/2024/02/...
1,2024-02-20,청주시 소규모 주택가 66곳에 재활용품 분리수거함 설치,https://n.news.naver.com/mnews/article/003/001...,충북 청주시는 소규모 주택가에 재활용품 분리수거함을 설치 지원한다고 20일 밝혔다....,https://imgnews.pstatic.net/image/003/2024/02/...
2,2024-02-20,'면사의 끝없는 부활' 제클린 업계 최초 우수 재활용 인증,https://n.news.naver.com/mnews/article/011/000...,기후테크 기업 제클린은 독자 개발한 재생 면화가 업계 최초의 우수 재활용 제품(GR...,https://imgnews.pstatic.net/image/011/2024/02/...
3,2024-02-20,정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해,https://n.news.naver.com/mnews/article/056/001...,오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망 재활용 업체에서 불이 나 5시간여...,https://imgnews.pstatic.net/image/056/2024/02/...
4,2024-02-20,전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”,https://n.news.naver.com/mnews/article/016/000...,전기차 폭발 유독성 화학물질 유출 등 배터리 안전문제가 대두되면서 범부처 ‘사용후 ...,https://imgnews.pstatic.net/image/016/2024/02/...
5,2024-02-20,하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실,https://n.news.naver.com/mnews/article/021/000...,‘소셜벤처 유니버시티’선순환 세계적 수준의 쓰레기 분리수거 제도를 시행하고 있는 우...,https://imgnews.pstatic.net/image/021/2024/02/...
6,2024-02-20,정읍 재활용시설 공장서 불…대응 1단계 발령,https://n.news.naver.com/mnews/article/003/001...,20일 오전 5시 전북 정읍시 소성면 용정리의 한 재활용시설 공장서 불이 났다. 소...,https://imgnews.pstatic.net/image/003/2024/02/...
7,2024-02-20,콘티넨탈 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차,https://n.news.naver.com/mnews/article/020/000...,콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 전략을 공개했다. 현...,https://imgnews.pstatic.net/image/020/2024/02/...
8,2024-02-20,울진군 자원재활용사업으로 탄소중립도시 실천,https://n.news.naver.com/mnews/article/277/000...,경북 울진군은 다양한 자원재활용 사업을 추진해 탄소중립 도시 정책에 군민들이 발맞추...,https://imgnews.pstatic.net/image/277/2024/02/...
9,2024-02-20,부산시설공단 녹지대 폐목재 재활용…농가에 무상 공급,https://n.news.naver.com/mnews/article/001/001...,부산시설공단은 도시고속도로 번영로 녹지대 관리 과정에서 발생하는 소나무 목재조각(우...,https://imgnews.pstatic.net/image/001/2024/02/...


In [62]:
news_df["content"]

0     [\n  \n\n\n\n현대차 인도네시아 공장에서 전기차 '아이오닉 5'가 생산되고...
1     [\n\n\n\n\n[청주=뉴시스] 충북 청주시청 제1임시청사. photo@news...
2     [\n\n\n\n\nGR인증 재생 면사로 제작된 제클린의 '리[Re;] 타올'/사진...
3     [\n\n\n\n\n[KBS 전주]오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망...
4     [\n\n\n\n\n전기차.[게티이미지뱅크][헤럴드경제=구본혁 ] 전기차 폭발, 유...
5     [\n\n\n\n\n‘소셜벤처 유니버시티’선순환세계적 수준의 쓰레기 분리수거 제도를...
6     [\n\n\n\n\n전북소방본부 제공 *재판매 및 DB 금지[정읍=뉴시스]최정규 기...
7     [\n\n\n\n\n콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 ...
8     [\n친환경 분리 배출 시스템으로 자원재활용 활성화 도모재활용품 자원화 시민참여 유...
9     [\n\n\n\n\n부산시설공단, 녹지대 폐목재 재활용[부산시설공단 제공]    (...
10    [\n\n\n\n\n매물로 나온 황조는 제강 분진을 재활용해 아연 함유량이 60% ...
11           [\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n[KBS 전주]\n]
12    [\n이규복 현대글로비스 대표선택과 집중으로 물류경쟁력·신사업 강화초대형 자동차운반...
13    [\n지질자원연구원, 폐리튬일차전지 침출액서 고순도 염화리튬 분리\n\n\n\n기술...
14    [\n\n\n\n\n/아모레퍼시픽 제공아모레퍼시픽그룹이 한국순환자원유통지원센터와 손...
15    [\n정부, 올해 전기차 '차종별 국비보조금' 공개배터리 에너지 밀도 등 고려 차등...
16    [\n\n\n\n\n나경수 SK지오센트릭의 사장“폐플라스틱은 재활용이 용이하도록 처...
17    [\n\n\n\n\n[서울=뉴시스]이상목 아모레퍼시픽그룹 대표이사(좌), 이

In [63]:
# 필요한 라이브러리를 다시 임포트합니다.
import pandas as pd
# "content" 컬럼에서 [, ], \n 문자 삭제
news_df["content"] = news_df["content"].str.replace("[\[\]\n]", "", regex=True)

# 변경된 내용 확인
news_df["content"]



0       현대차 인도네시아 공장에서 전기차 '아이오닉 5'가 생산되고 있다. 사진 현대자...
1     청주=뉴시스 충북 청주시청 제1임시청사. photo@newsis.com *재판매 및...
2     GR인증 재생 면사로 제작된 제클린의 '리Re; 타올'/사진 제공=제클린서울경제 기...
3     KBS 전주오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망 재활용 업체에서 불이 ...
4     전기차.게티이미지뱅크헤럴드경제=구본혁  전기차 폭발, 유독성 화학물질 유출 등 배터...
5     ‘소셜벤처 유니버시티’선순환세계적 수준의 쓰레기 분리수거 제도를 시행하고 있는 우리...
6     전북소방본부 제공 *재판매 및 DB 금지정읍=뉴시스최정규 기자 = 20일 오전 5시...
7     콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 전략을 공개했다.현재...
8     친환경 분리 배출 시스템으로 자원재활용 활성화 도모재활용품 자원화 시민참여 유도, ...
9     부산시설공단, 녹지대 폐목재 재활용부산시설공단 제공    (부산=연합뉴스) 조정호 ...
10    매물로 나온 황조는 제강 분진을 재활용해 아연 함유량이 60% 수준인 조선화아연을 ...
11                                               KBS 전주
12    이규복 현대글로비스 대표선택과 집중으로 물류경쟁력·신사업 강화초대형 자동차운반선 3...
13    지질자원연구원, 폐리튬일차전지 침출액서 고순도 염화리튬 분리기술 모식도한국지질자원연...
14    /아모레퍼시픽 제공아모레퍼시픽그룹이 한국순환자원유통지원센터와 손잡고 화장품 유리병의...
15    정부, 올해 전기차 '차종별 국비보조금' 공개배터리 에너지 밀도 등 고려 차등 지급...
16    나경수 SK지오센트릭의 사장“폐플라스틱은 재활용이 용이하도록 처리하는 '굿바이' 활...
17    서울=뉴시스이상목 아모레퍼시픽그룹 대표이사(좌), 이명환 한국순환자원유통센터

In [64]:
# 'news_df'의 "content" 컬럼을 'df_articles'의 "content" 컬럼에 저장
df_articles["content"] = news_df["content"]
# 저장된 내용 확인
df_articles



,date,title,link,content,imageUrl
0,2024-02-20,테슬라 모델Y 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓,https://n.news.naver.com/mnews/article/025/000...,현대차 인도네시아 공장에서 전기차 '아이오닉 5'가 생산되고 있다. 사진 현대자...,https://imgnews.pstatic.net/image/025/2024/02/...
1,2024-02-20,청주시 소규모 주택가 66곳에 재활용품 분리수거함 설치,https://n.news.naver.com/mnews/article/003/001...,청주=뉴시스 충북 청주시청 제1임시청사. photo@newsis.com *재판매 및...,https://imgnews.pstatic.net/image/003/2024/02/...
2,2024-02-20,'면사의 끝없는 부활' 제클린 업계 최초 우수 재활용 인증,https://n.news.naver.com/mnews/article/011/000...,GR인증 재생 면사로 제작된 제클린의 '리Re; 타올'/사진 제공=제클린서울경제 기...,https://imgnews.pstatic.net/image/011/2024/02/...
3,2024-02-20,정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해,https://n.news.naver.com/mnews/article/056/001...,KBS 전주오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망 재활용 업체에서 불이 ...,https://imgnews.pstatic.net/image/056/2024/02/...
4,2024-02-20,전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”,https://n.news.naver.com/mnews/article/016/000...,"전기차.게티이미지뱅크헤럴드경제=구본혁 전기차 폭발, 유독성 화학물질 유출 등 배터...",https://imgnews.pstatic.net/image/016/2024/02/...
5,2024-02-20,하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실,https://n.news.naver.com/mnews/article/021/000...,‘소셜벤처 유니버시티’선순환세계적 수준의 쓰레기 분리수거 제도를 시행하고 있는 우리...,https://imgnews.pstatic.net/image/021/2024/02/...
6,2024-02-20,정읍 재활용시설 공장서 불…대응 1단계 발령,https://n.news.naver.com/mnews/article/003/001...,전북소방본부 제공 *재판매 및 DB 금지정읍=뉴시스최정규 기자 = 20일 오전 5시...,https://imgnews.pstatic.net/image/003/2024/02/...
7,2024-02-20,콘티넨탈 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차,https://n.news.naver.com/mnews/article/020/000...,콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 전략을 공개했다.현재...,https://imgnews.pstatic.net/image/020/2024/02/...
8,2024-02-20,울진군 자원재활용사업으로 탄소중립도시 실천,https://n.news.naver.com/mnews/article/277/000...,"친환경 분리 배출 시스템으로 자원재활용 활성화 도모재활용품 자원화 시민참여 유도, ...",https://imgnews.pstatic.net/image/277/2024/02/...
9,2024-02-20,부산시설공단 녹지대 폐목재 재활용…농가에 무상 공급,https://n.news.naver.com/mnews/article/001/001...,"부산시설공단, 녹지대 폐목재 재활용부산시설공단 제공 (부산=연합뉴스) 조정호 ...",https://imgnews.pstatic.net/image/001/2024/02/...


In [65]:
df_articles['content'] = df_articles['content'].str.replace(',', '')
df_articles

,date,title,link,content,imageUrl
0,2024-02-20,테슬라 모델Y 보조금 3분의1 토막…재활용 힘든 中배터리 쓴 탓,https://n.news.naver.com/mnews/article/025/000...,현대차 인도네시아 공장에서 전기차 '아이오닉 5'가 생산되고 있다. 사진 현대자...,https://imgnews.pstatic.net/image/025/2024/02/...
1,2024-02-20,청주시 소규모 주택가 66곳에 재활용품 분리수거함 설치,https://n.news.naver.com/mnews/article/003/001...,청주=뉴시스 충북 청주시청 제1임시청사. photo@newsis.com *재판매 및...,https://imgnews.pstatic.net/image/003/2024/02/...
2,2024-02-20,'면사의 끝없는 부활' 제클린 업계 최초 우수 재활용 인증,https://n.news.naver.com/mnews/article/011/000...,GR인증 재생 면사로 제작된 제클린의 '리Re; 타올'/사진 제공=제클린서울경제 기...,https://imgnews.pstatic.net/image/011/2024/02/...
3,2024-02-20,정읍 재활용업체 불 5시간 만에 꺼져…2억여 원 재산 피해,https://n.news.naver.com/mnews/article/056/001...,KBS 전주오늘(20) 새벽 5시쯤 정읍시 소성면 한 폐어망 재활용 업체에서 불이 ...,https://imgnews.pstatic.net/image/056/2024/02/...
4,2024-02-20,전기車 폐배터리 재활용…STEPI “통합관리체계 구축 서둘러야”,https://n.news.naver.com/mnews/article/016/000...,전기차.게티이미지뱅크헤럴드경제=구본혁 전기차 폭발 유독성 화학물질 유출 등 배터리...,https://imgnews.pstatic.net/image/016/2024/02/...
5,2024-02-20,하나금융 ‘청년 스타트업 지원’ 덕분… ‘NAWA’ 일회용컵 재활용사업 결실,https://n.news.naver.com/mnews/article/021/000...,‘소셜벤처 유니버시티’선순환세계적 수준의 쓰레기 분리수거 제도를 시행하고 있는 우리...,https://imgnews.pstatic.net/image/021/2024/02/...
6,2024-02-20,정읍 재활용시설 공장서 불…대응 1단계 발령,https://n.news.naver.com/mnews/article/003/001...,전북소방본부 제공 *재판매 및 DB 금지정읍=뉴시스최정규 기자 = 20일 오전 5시...,https://imgnews.pstatic.net/image/003/2024/02/...
7,2024-02-20,콘티넨탈 페트병으로 타이어 만든다… 지속가능성 위해 자원 재활용 박차,https://n.news.naver.com/mnews/article/020/000...,콘티넨탈은 20일 주력 사업 중 하나인 지속가능성을 향한 미래 전략을 공개했다.현재...,https://imgnews.pstatic.net/image/020/2024/02/...
8,2024-02-20,울진군 자원재활용사업으로 탄소중립도시 실천,https://n.news.naver.com/mnews/article/277/000...,친환경 분리 배출 시스템으로 자원재활용 활성화 도모재활용품 자원화 시민참여 유도 영...,https://imgnews.pstatic.net/image/277/2024/02/...
9,2024-02-20,부산시설공단 녹지대 폐목재 재활용…농가에 무상 공급,https://n.news.naver.com/mnews/article/001/001...,부산시설공단 녹지대 폐목재 재활용부산시설공단 제공 (부산=연합뉴스) 조정호 기...,https://imgnews.pstatic.net/image/001/2024/02/...


In [66]:
df_articles.to_csv('C:/Users/20161/Downloads/articles_data_final.csv',encoding='utf-8-sig',index=False)

In [ ]:
import 

In [ ]:
z